In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gdown
import pandas as pd

file_id = "1EKTIVcSr2LUMZo9ucJJqvhCCNC7Zi68G"

url = f"https://drive.google.com/uc?id={file_id}"

output_path = "train_df_sample.pkl"
gdown.download(url, output_path, quiet=False)

df = pd.read_pickle(output_path)

df.head()

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

warnings.simplefilter(action='ignore', category=FutureWarning)

Downloading...
From (original): https://drive.google.com/uc?id=1EKTIVcSr2LUMZo9ucJJqvhCCNC7Zi68G
From (redirected): https://drive.google.com/uc?id=1EKTIVcSr2LUMZo9ucJJqvhCCNC7Zi68G&confirm=t&uuid=ca4f379c-fda2-4c9f-b288-44003476df52
To: /content/train_df_sample.pkl
100%|██████████| 334M/334M [00:02<00:00, 136MB/s]


> ### 데이터 준비

In [2]:
warnings.filterwarnings('ignore', module='sklearn.metrics.cluster')

In [3]:
def drop_null_cols(df, threshold=0.8):
    """
    데이터프레임에서 결측치 비율이 threshold 이상인 변수를 제거하는 함수
    """
    null_percent = df.isnull().mean()
    drop_cols = list(null_percent[null_percent >= threshold].index)
    df = df.drop(drop_cols, axis=1)
    print(f"Dropped {len(drop_cols)} columns: {', '.join(drop_cols)}")
    return df

In [7]:
df = df.reset_index()

import hashlib

def encode_customer_id(id_str):
    encoded_id = hashlib.sha256(id_str.encode('utf-8')).hexdigest()[:16]
    return encoded_id

df['customer_ID'] = df['customer_ID'].apply(encode_customer_id)
df = drop_null_cols(df)

Dropped 106 columns: D_49_mean, D_49_std, D_49_min, D_49_max, D_49_last, D_73_mean, D_73_std, D_73_min, D_73_max, D_73_last, D_76_mean, D_76_std, D_76_min, D_76_max, D_76_last, R_9_mean, R_9_std, R_9_min, R_9_max, R_9_last, B_29_mean, B_29_std, B_29_min, B_29_max, B_29_last, D_87_mean, D_87_std, D_87_min, D_87_max, D_87_last, D_88_mean, D_88_std, D_88_min, D_88_max, D_88_last, D_106_mean, D_106_std, D_106_min, D_106_max, D_106_last, R_26_mean, R_26_std, R_26_min, R_26_max, R_26_last, D_108_mean, D_108_std, D_108_min, D_108_max, D_108_last, D_110_mean, D_110_std, D_110_min, D_110_max, D_110_last, D_111_mean, D_111_std, D_111_min, D_111_max, D_111_last, B_39_mean, B_39_std, B_39_min, B_39_max, B_39_last, B_42_mean, B_42_std, B_42_min, B_42_max, B_42_last, D_132_mean, D_132_std, D_132_min, D_132_max, D_132_last, D_134_mean, D_134_std, D_134_min, D_134_max, D_134_last, D_135_mean, D_135_std, D_135_min, D_135_max, D_135_last, D_136_mean, D_136_std, D_136_min, D_136_max, D_136_last, D_137_me

In [8]:
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_68"
]
cat_features = [f"{cf}_last" for cf in cat_features]

In [9]:
import random

num_cols = df.select_dtypes(include=np.number).columns.tolist()
num_cols = [col for col in num_cols if 'target' not in col and col not in cat_features]
num_cols_sample = random.sample([col for col in num_cols if 'target' not in col], 100)

In [10]:
feature_list = num_cols_sample + cat_features
all_list = feature_list + ['target']

In [11]:
df = df[all_list]

In [17]:
for categorical_feature in cat_features:
    if df[categorical_feature].dtype == 'float16':
        df.loc[:, categorical_feature] = df[categorical_feature].astype(str)
    elif df[categorical_feature].dtype == 'category':
        df.loc[:, categorical_feature] = df[categorical_feature].astype(str)
    elif df[categorical_feature].dtype == 'object':
        df.loc[:, categorical_feature] = df[categorical_feature].astype(str)


In [18]:
from sklearn.preprocessing import LabelEncoder

le_encoder = LabelEncoder()

for categorical_feature in cat_features:
    df[categorical_feature].fillna(value='NaN', inplace=True)
    df[categorical_feature] = le_encoder.fit_transform(df[categorical_feature])

In [19]:
from sklearn.impute import SimpleImputer

def impute_nan(df, num_cols, strategy='mean'):
    """
    NaN 값을 strategy에 따라 num_cols에 대해 impute하는 함수

    :param df: DataFrame
    :param num_cols: list, imputation 대상 numeric column 리스트
    :param strategy: str, imputation 전략 (default: 'mean')
    :return: DataFrame, imputed DataFrame
    """
    imputer = SimpleImputer(strategy=strategy)
    df[num_cols] = imputer.fit_transform(df[num_cols])
    return df

In [20]:
df = impute_nan(df,num_cols_sample, strategy="mean")

> #### taod 패키지 설치

In [21]:
%pip install toad

> #### IV값을 기준으로 변수를 선택해보자. (Feature Selection)

In [22]:
import toad

# 타겟 변수 정의
target = 'target'

# 정보값(IV) 계산 및 특성 선택
iv_df = toad.quality(df, target=target, iv_only=True)
selected_features = iv_df[iv_df['iv'] > 0.1].index  # 'name' 대신 'index'를 사용합니다.

# WOE 변환
trans = toad.transform.WOETransformer()
df_woe = trans.fit_transform(df[selected_features], df[target])

# 이제 df_woe를 추가적인 모델링에 사용할 수 있습니다.


toad는 파이썬의 데이터 전처리 및 탐색을 위한 오픈 소스 패키지입니다. 주요 기능은 다음과 같습니다:

데이터 품질 평가: toad.quality() 함수를 사용하여 변수별 정보값(IV, Information Value)을 계산하고, 타겟 변수와의 상관 관계를 평가합니다. 이를 통해 변수의 중요도를 판단하고 특성 선택에 활용할 수 있습니다.

데이터 변환: WOE (Weight of Evidence) 변환을 위한 toad.transform.WOETransformer()를 사용할 수 있습니다. WOE는 범주형 변수나 연속형 변수를 범주화하여 정보값을 측정하는 방법으로, 변수의 예측력을 개선하고 모델 성능을 향상시킬 수 있습니다.

In [23]:
df_woe

,D_48_last,D_48_max,R_1_max,D_61_max,B_11_last,B_7_max,D_62_mean,B_38_last,B_33_min,B_20_max,...,D_120_count,D_116_count,D_122_max,D_113_last,D_117_last,D_112_max,D_39_min,D_72_min,R_25_max,R_25_mean
0,1.096556,0.215811,1.042489,0.979969,0.572486,-0.056123,0.754807,1.408741,0.908958,-1.050746,...,0.500820,0.500820,0.685814,-0.404430,0.192825,-0.231197,0.572486,0.637024,0.531664,0.077408
1,1.378961,0.717067,0.601933,1.042489,-0.903421,-2.324807,-1.631660,0.394101,1.630276,-1.665561,...,-0.212060,-0.212060,-0.444646,-1.208803,-0.421929,-0.231197,0.349342,-0.210274,-0.119435,-0.242023
2,-0.903421,-0.749270,1.735636,-1.154735,1.735636,-1.154735,-1.901950,-1.277605,-0.328538,1.042489,...,0.371257,0.371257,0.371257,0.371257,0.371257,0.171843,-0.343805,1.042489,1.042489,-0.229142
3,-1.665561,-0.423848,-1.585796,-1.154735,-1.596568,-2.048553,-1.260096,-1.873590,-1.915386,-1.770922,...,-0.212060,-0.212060,-0.712131,-0.056123,-0.751712,-0.231197,-0.343805,0.674764,0.035896,-0.021337
4,-0.903421,-1.596568,-2.029432,-1.901950,0.231559,-2.568429,-0.749270,-1.873590,-2.337625,-1.659990,...,-0.212060,-0.212060,-0.163268,-0.566949,0.192825,-0.114553,0.706017,-0.056123,0.001035,-0.138361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.637024,1.340324,3.452433,1.385198,1.998001,-1.208803,1.265633,0.394101,-0.566949,1.072258,...,-0.212060,-0.212060,-0.550324,-0.392595,0.467881,-0.231197,-0.566949,-0.972414,-0.010661,0.036967
99996,-1.665561,-1.036952,-1.382314,-0.515655,-0.903421,-2.253348,-1.596568,-1.873590,-2.228483,-1.408516,...,-0.212060,-0.212060,0.531664,-0.662259,-0.421929,-0.231197,0.754807,0.516396,-0.119435,0.419546
99997,-2.324807,-1.847883,-1.516029,-2.289715,-2.512859,0.908958,-1.355406,-1.277605,0.349342,-0.013564,...,-0.212060,-0.212060,-0.444646,0.240974,0.128533,-0.114553,-1.442418,-0.497956,-0.042965,-0.178013
99998,-1.036952,-0.566949,-1.407254,-1.015899,-1.901950,-1.308886,-2.621073,-1.873590,0.754807,-1.260096,...,-0.212060,-0.212060,-0.627926,-0.056123,-0.421929,-0.231197,0.215811,-0.903421,-0.267706,-0.023862


> #### 선택된 피처를 기반으로 binning 작업을 진행한다. 이번엔 방법을 조금 다르게해서, 각 구간이 최소 5%의 데이터를 가지도록 구간화를 수행한다.

In [25]:
combiner = toad.transform.Combiner()
combiner.fit(df[selected_features], y = df[target], method = 'chi', min_samples = 0.05)
#min_samples = 0.05로 설정하면, 각 구간이 최소 5%의 데이터를 가지도록 구간화를 수행합니다.

# 구간화 결과를 살펴봅시다.
binning_result = combiner.export()

> #### iv=True는 toad.plot 함수의 매개변수 중 하나입니다. 이 매개변수를 True로 설정하면, 그래프에 Information Value (IV) 값을 표시합니다.

In [26]:
toad.detect(df)[:10]

,type,size,missing,unique,mean_or_top1,std_or_top2,min_or_top3,1%_or_top4,10%_or_top5,50%_or_bottom5,75%_or_bottom4,90%_or_bottom3,99%_or_bottom2,max_or_bottom1
R_5_max,float64,100000,0.00%,1954,0.160710,0.647100,2.115965e-05,0.004654,0.008186,0.009529,0.009865,0.504395,3.007812,20.515625
B_36_min,float64,100000,0.00%,8277,0.001142,0.007943,0.000000e+00,0.000008,0.000086,0.000570,0.001143,0.001955,0.005966,0.355469
R_1_max,float64,100000,0.00%,4691,0.275044,0.425598,6.556511e-07,0.005035,0.008499,0.009865,0.506348,1.001953,1.755859,3.005859
B_21_max,float64,100000,0.00%,5530,0.168623,1.430895,3.278255e-06,0.004631,0.008110,0.009476,0.009804,0.009956,4.453125,81.062500
D_125_mean,float64,100000,0.00%,3840,0.088981,0.221105,8.940697e-07,0.002394,0.003975,0.005268,0.006336,0.379175,1.005859,3.544922
D_48_last,float64,100000,0.00%,10300,0.390582,0.334140,-9.597778e-03,0.000301,0.022705,0.330322,0.684204,0.913574,1.039062,3.619141
R_25_mean,float64,100000,0.00%,2923,0.008298,0.026612,1.293421e-05,0.002745,0.003902,0.005024,0.005627,0.006229,0.104127,1.002930
R_7_last,float64,100000,0.00%,10585,0.166280,2.346192,4.172325e-07,0.000109,0.001050,0.005264,0.007889,0.009483,2.578203,324.000000
B_22_mean,float64,100000,0.00%,4562,0.103926,0.184312,3.099442e-06,0.002892,0.004093,0.005569,0.121155,0.427002,0.773926,2.503906
D_113_last,float64,100000,0.00%,6429,0.157087,0.224938,5.960464e-08,0.000165,0.001771,0.008850,0.207642,0.408447,1.001953,7.007812


> #### toad.detect(df)의 결과는 변수 이름과 해당 변수의 데이터 유형을 포함한 데이터프레임으로 반환됩니다. 결과에서 첫 10개의 변수와 그에 해당하는 데이터 유형을 확인할 수 있습니다.

In [27]:
toad.quality(df,'target',iv_only=True)[:15]

,iv,gini,entropy,unique
D_48_last,2.412339,NaN,NaN,10300.0
D_48_max,2.008316,NaN,NaN,7339.0
R_1_max,1.889594,NaN,NaN,4691.0
D_61_max,1.849952,NaN,NaN,7325.0
B_11_last,1.794553,NaN,NaN,12569.0
B_7_max,1.729111,NaN,NaN,6852.0
D_62_mean,1.722439,NaN,NaN,8297.0
B_38_last,1.682357,NaN,NaN,8.0
B_33_min,1.422738,NaN,NaN,8321.0
B_20_max,1.397078,NaN,NaN,2454.0


> #### 각 변수별 IV값을 살펴보자.

#### Feature Selection

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">

<h3 align="left"><font color='#DEB887'>💡 Notes:</font></h3>

* empty=0.9: 결측치 비율이 90%보다 큰 특성들은 필터링됩니다.
* iv=0.02: IV(Information Value)가 0.02보다 작은 특성들은 제거됩니다.
* corr=0.7: 두 개 이상의 특성들 간의 피어슨 상관계수가 0.7보다 큰 경우, IV가 더 낮은 특성들이 제거됩니다.
* return_drop=False: True로 설정하면, 함수는 삭제된 열들의 리스트를 반환합니다.
* exclude=None: 알고리즘에서 제외할 특성들의 리스트를 입력합니다. 일반적으로 ID 열이나 월(Month) 열 등이 해당됩니다.

In [28]:
train_selected, dropped = toad.selection.select(df,target = 'target', empty = 0.5, iv = 0.05, corr = 0.7, return_drop=True, exclude=['D_117_last'])
print(dropped)
print(train_selected.shape)

{'empty': array([], dtype=float64), 'iv': array(['B_36_min', 'D_89_min', 'D_128_std', 'S_18_max', 'D_63_nunique',
       'D_86_std', 'D_82_mean', 'S_19_mean', 'S_17_min', 'D_144_min',
       'R_8_min', 'D_144_max', 'D_139_std', 'R_12_max', 'D_93_std',
       'S_20_min', 'D_145_std', 'D_116_last', 'D_126_last', 'D_63_last'],
      dtype=object), 'corr': array(['D_89_std', 'B_22_mean', 'R_13_std', 'R_4_std', 'B_4_max',
       'B_20_max', 'D_48_max', 'R_25_mean', 'D_113_last', 'D_116_count',
       'D_118_mean', 'R_24_std', 'B_14_max', 'D_53_std', 'R_2_max',
       'S_7_last', 'S_25_min', 'D_58_max', 'D_77_mean'], dtype=object)}
(100000, 72)


> #### toad.selection.select 함수는 선택된 특성들로 이루어진 훈련 데이터셋인 train_selected와 삭제된 열들의 리스트인 dropped를 반환합니다. 또한, train_selected.shape는 선택된 특성들을 포함하는 훈련 데이터셋의 크기를 출력합니다.

In [29]:
# initialise
c = toad.transform.Combiner()


# Train binning with the selected features from previous; use reliable Chi-squared binning, and control that each bucket has at least 5% sample.
c.fit(train_selected, y = 'target', method = 'chi', min_samples = 0.05, exclude = ['D_117_last'])

In [30]:
print('D_59_min:',c.export()['D_59_min'])
print('R_15_std:',c.export()['R_15_std'])
print('S_3_last:',c.export()['S_3_last'])

KeyError: 'D_59_min'

> ##### 위 코드는 c.export()를 사용하여 binning 결과 중 특정 변수들의 정보를 출력하는 부분입니다.
c.export(): Combiner 객체 c의 binning 결과를 딕셔너리 형태로 반환합니다. 딕셔너리의 키는 각 변수의 이름이며, 값은 해당 변수의 binning 정보를 담고 있는 객체입니다.
각 변수에 대한 binning 정보는 해당 변수의 구간(bin)과 해당 구간의 라벨(label) 등을 포함하고 있습니다. 출력된 정보를 통해 각 변수의 binning 결과를 확인할 수 있습니다.

In [ ]:
from toad.plot import bin_plot

# 학습 데이터(train_selected)에서 'var_d2' 변수의 bin 결과를 확인합니다.
col = 'D_59_min'

# 시각화를 위해 'labels = True'로 설정하는 것이 좋습니다.
bin_plot(c.transform(train_selected[[col, 'target']], labels=True), x=col, target='target')


> ##### 위 코드는 bin_plot 함수를 사용하여 'R_4_std' 변수의 binning 결과를 시각화하는 부분입니다.

> ##### c.transform(train_selected[[col, 'target']], labels=True): 학습 데이터(train_selected)의 'R_4_std' 변수와 목표 변수('target')를 선택하여 binning 결과를 생성합니다. labels=True로 설정하여 bin의 라벨을 포함한 결과를 반환합니다.

> ##### bin_plot(...): binning 결과를 시각화합니다. x축에는 'R_4_std' 변수의 값이 나타나며, target 변수('target')의 값에 따라 각 bin의 분포를 확인할 수 있습니다. bin_plot 함수를 통해 binning 결과를 시각적으로 확인하여 데이터의 패턴을 파악할 수 있습니다.

In [ ]:
# 학습 데이터(train_selected)에서 'D_51_min' 변수의 bin 결과를 확인합니다.
col = 'S_3_last'

# 범주형 변수의 경우 'labels = True'로 설정하는 것이 좋습니다.
bin_plot(c.transform(train_selected[[col, 'target']], labels=True), x=col, target='target')

Toad의 binning 기능은 범주형 및 수치형 변수를 모두 지원합니다. "toad.transform.Combiner()" 클래스를 사용하여 학습하며, 절차는 다음과 같습니다:

* 초기화(initalise) : c = toad.transform.Combiner()
* *train binning*: c.fit(dataframe, y='target', method='chi', min_samples=None, n_bins=None, empty_separate=False)
* y: 목표 변수;
* method: binning에 적용할 방법. 'chi' (카이제곱), 'dt' (의사결정 트리), 'kmeans' (K-means), 'quantile' (동일한 백분위수 기준), 'step' (동일한 간격)을 지원합니다.
* min_samples: 샘플당 요구되는 최소 수 또는 비율. 각 버킷에 필요한 최소 샘플 수 / 비율입니다.
* n_bins: 최소한의 버킷 수. 수가 너무 큰 경우, 알고리즘은 얻을 수 있는 최대 버킷 수를 반환합니다.
* empty_separate: 누락된 값이 버킷 내에 별도로 분리되는지 여부. False인 경우, 누락된 값은 가장 가까운 나쁜 비율 버킷과 함께 배치됩니다.
* binning 결과: c.export()
* bins 조정: c.update(dict)
* bins 적용 및 이산값으로 변환: c.transform(dataframe, labels=False)
* labels: 데이터를 설명 라벨로 변환할지 여부. False인 경우 0, 1, 2...로 반환됩니다. 범주형 변수는 비율의 내림차순으로 정렬됩니다. True인 경우 (-무한대, 0], (0, 10], (10, 무한대)와 같이 반환됩니다.

* 참고: 1. 불필요한 열을 제외하는 것을 잊지 마세요. 특히 ID 열과 타임스탬프 열은 제외해야 합니다.. 2. 고유 값이 많은 열은 학습에 많은 시간이 소요될 수 있습니다.*

In [ ]:
# 초기화
transer = toad.transform.WOETransformer()

# transer.fit_transform() 및 combiner.transform()을 적용합니다. target을 제외하도록 주의하세요.
train_woe = transer.fit_transform(c.transform(train_selected), train_selected['target'], exclude='target')

train_woe.head(3)


* WOETransformer를 초기화합니다.
* combiner.transform()의 결과에 transer.fit_transform()을 적용합니다.
* target을 제외하도록 주의하세요.
* 변환된 train_woe 데이터프레임의 처음 3개 행을 출력합니다.

In [ ]:
col = train_woe.columns.tolist()[:-1]
col.remove('D_117_last')

> #### 예시를 위해 제거했던 변수를 제거해줍니다.

> #### 예시를 위해 간단한 로지스틱 회귀 모델을 적합해봅니다.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_woe[col], train_woe['target'])

# 훈련 데이터와 Out-of-Time (OOT) 데이터에 대해 예측된 확률을 구합니다.
pred_train = lr.predict_proba(train_woe[col])[:,1]

> #### Toad 역시 쉽게 KS 통계량과 AUC를 계산할 수 있는 라이브러리를 제공합니다.

In [ ]:
from toad.metrics import KS, AUC

In [ ]:
print('train KS',KS(pred_train, train_woe['target']))
print('train AUC',AUC(pred_train, train_woe['target']))

> #### Toad에서도 OptBinning과 마찬가지로 스코어링을 할 수 있는 스코어 카드 기능을 제공합니다.

In [ ]:
card = toad.ScoreCard(
    combiner = c,
    transer = transer,
    C=0.1,
    base_score = 600,
    base_odds = 35 ,
    pdo = 60,
    rate = 2
)

card.fit(train_woe[col], train_woe['target'])

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'>💡 Note!:</font></h3>



* toad.ScoreCard는 Scorecard 모델을 생성하기 위한 클래스입니다.

* combiner는 binning 결과를 담고 있는 toad.transform.Combiner 객체를 전달합니다.

* transer는 WOE 변환을 담당하는 toad.transform.WOETransformer 객체를 전달합니다.

* C는 Logistic Regression에서의 규제 강도를 나타내는 매개변수입니다.

* base_score는 기준 스코어로, 기본적으로 모든 변수의 WOE 값이 0일 때의 스코어입니다.

* base_odds는 기준 오즈로, 기본적으로 모든 변수의 WOE 값이 0일 때의 오즈입니다.

* pdo는 Point to Double the Odds로, 오즈를 두배로 만들기 위해 필요한 점수의 차이입니다.

* rate는 모델의 점수 스케일을 조절하기 위한 비율입니다.

* fit() 메서드를 사용하여 Scorecard 모델을 훈련시킵니다. train_woe[col]은 독립 변수를, train_woe['target']은 종속 변수를 나타냅니다.

In [ ]:
sample_train_woe = train_woe.sample(3)
score_sample = card.predict(sample_train_woe[col])
score_sample_rounded = score_sample.round().astype(int)

for i, score in enumerate(score_sample_rounded, start=1):
    print(f"{i}번째 고객의 점수는 \033[1;34m{score}\033[0m점 입니다.")


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'>💡 Note:</font></h3>


* 1. `sample_train_woe = train_woe.sample(3)`: `sample()` 함수를 사용하여 `train_woe` 데이터셋에서 임의로 3개의 샘플을 추출합니다. 이 샘플들은 점수를 계산할 데이터입니다.

* 2. `score_sample = card.predict(sample_train_woe[col])`: `card` 라는 ScoreCard 객체의 `predict()` 메서드를 사용하여 위에서 추출한 샘플에 대한 점수를 계산합니다. 이 때, `sample_train_woe[col]`을 통해 샘플의 해당 컬럼(변수)만 사용하게 됩니다.

* 3. `score_sample_rounded = score_sample.round().astype(int)`: 계산된 점수는 일반적으로 소수점 형태로 반환됩니다. 이 줄의 코드는 `round()` 함수를 사용하여 계산된 점수를 반올림하고, `astype(int)`를 통해 정수형으로 변환합니다.

* 4. `for i, score in enumerate(score_sample_rounded, start=1)`: 이 줄은 반복문을 사용하여 각 샘플에 대한 점수를 순서대로 출력합니다. `enumerate()` 함수는 반복 가능한 객체(여기서는 `score_sample_rounded`)를 입력으로 받아 인덱스 번호(i)와 그에 해당하는 값을(score) 함께 반환합니다. `start=1`은 인덱스 번호가 1부터 시작하도록 설정합니다.

* 5. `print(f"{i}번째 고객의 점수는 \033[1;34m{score}\033[0m점 입니다.")`: 이 줄은 문자열 포매팅을 사용하여 각 샘플에 대한 점수를 보기 좋게 출력합니다. `\033[1;34m`와 `\033[0m` 사이에 있는 텍스트는 파란색으로 출력됩니다.

In [ ]:
card.export()

> ##### card.export()는 훈련된 Scorecard 모델의 정보를 추출하는 메서드입니다. 이 메서드를 호출하면 다음과 같은 정보를 포함하는 딕셔너리를 반환합니다:

> 'model_info': Scorecard 모델의 기본 정보를 담고 있는 딕셔너리입니다. 여기에는 C, base_score, base_odds, pdo, rate 등의 모델 설정값이 포함됩니다.

> 'features': Scorecard 모델에 사용된 독립 변수의 정보를 담고 있는 딕셔너리입니다. 각 변수마다 변수명, WOE 정보, 점수 정보 등이 포함됩니다.
이를 통해 훈련된 Scorecard 모델의 설정값과 각 변수의 정보를 확인할 수 있습니다.